In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [4]:
print len(X_train1), len(X_valid1), len(X_test1)

28038 2558 5139


Shape of inputs/outputs

In [21]:
n_inputs = 28*28  # MNIST
n_outputs = 5

In [38]:
tf.reset_default_graph()

In [39]:
X = tf.placeholder(shape=[None, n_inputs], dtype=tf.float32, name='X')
y = tf.placeholder(shape=None, dtype=tf.int64, name='X')

**Deep Neural Network Structure:**

In [40]:
# number of neurons by layer:
n_neur = 100

hidden_1 = tf.layers.dense(inputs=X,
                           units=n_neur,
                           kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                           activation=tf.nn.elu,
                           name="hidden1")
hidden_2 = tf.layers.dense(inputs=hidden_1,
                           units=n_neur,
                           kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                           activation=tf.nn.elu,
                           name="hidden2")
hidden_3 = tf.layers.dense(inputs=hidden_2,
                           units=n_neur,
                           kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                           activation=tf.nn.elu,
                           name="hidden3")
hidden_4 = tf.layers.dense(inputs=hidden_3,
                           units=n_neur,
                           kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                           activation=tf.nn.elu,
                           name="hidden4")
hidden_5 = tf.layers.dense(inputs=hidden_4,
                           units=n_neur,
                           kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                           activation=tf.nn.elu,
                           name="hidden5")
logits = tf.layers.dense(inputs=hidden_5,
                         units=n_outputs,
                         kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                         name='logits')

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

**Optimization:**

In [41]:
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                   beta1=0.9,
                                   beta2=0.999)
training_step = optimizer.minimize(loss, name='training_step')
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
saver = tf.train.Saver()

**Training**

In [42]:
n_epochs = 1000
batch_size = 50
stopping_threshold = 20
iter_since_best = 0
best_loss_val = float('inf')

In [43]:
n_samples = X_train1.shape[0]

In [44]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        rand_idx = np.random.permutation(np.arange(n_samples))
        for iteration in range(n_samples // batch_size):
            batch_idx = rand_idx[(iteration*batch_size):((iteration+1)*batch_size)]
            X_batch = X_train1[batch_idx,:]
            y_batch = y_train1[batch_idx]
            sess.run(training_step, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid1,
                                           y: y_valid1})
        loss_val = loss.eval(feed_dict={X: X_valid1,
                                           y: y_valid1})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val, )
        if epoch % 10 == 0:
            if loss_val < best_loss_val:
                save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
                best_loss_val = loss_val
                iter_since_best = 0
        if iter_since_best >= stopping_threshold:
            break
        iter_since_best +=1
with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

(0, 'Train accuracy:', 1.0, 'Val accuracy:', 0.97849882)
(1, 'Train accuracy:', 0.98000002, 'Val accuracy:', 0.98592651)
(2, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98827207)
(3, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98827207)
(4, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98788118)
(5, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98866302)
(6, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98827207)
(7, 'Train accuracy:', 1.0, 'Val accuracy:', 0.99179047)
(8, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98397183)
(9, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98944485)
(10, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98358095)
(11, 'Train accuracy:', 1.0, 'Val accuracy:', 0.99296325)
(12, 'Train accuracy:', 1.0, 'Val accuracy:', 0.99179047)
(13, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98631746)
(14, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98749024)
(15, 'Train accuracy:', 1.0, 'Val accuracy:', 0.99257231)
(16, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98944485)
(17, 'Train accur